In [1]:
import matplotlib.pyplot as plt
import cv2
import json
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torchvision.transforms import Compose, RandomHorizontalFlip, RandomRotation, RandomResizedCrop, ColorJitter, RandomPerspective, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from PIL import Image
from progressbar import ProgressBar
import time
from multiprocessing import Pool
from functools import partial
import random
from sklearn.model_selection import train_test_split

In [2]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        #x = torch.Tensor(self.features[idx])
        #y = torch.Tensor(self.targets[idx])
        x = self.X[idx]
        y = self.y[idx]
        
        return x,y

In [3]:
def resize_256_256(inp):
    return(F.interpolate(inp, size=(256,256), mode="bilinear", align_corners=False))

In [4]:
main_path = r"C:/Users/frede/Desktop/Bachelorprojekt/datasets/midair/MidAir/Kite_training/"

In [5]:
amount_images = 15000
amount_trajectory = 30
total_trajectory = amount_trajectory*3
image_per_trajectory = round(amount_images/total_trajectory)
print(image_per_trajectory)

167


In [6]:
def create_data_midair(amount_images, path):
    
    amount_trajectory = 30
    total_trajectory = amount_trajectory*3
    image_per_trajectory = round(amount_images/total_trajectory)

    
    np.random.seed(1337)
    combine_images = []
    combine_depth_images = []
    combine_depth_images_no_rgb = []


    #Loop over the three weather types ["cloudy", "sunny", "sunset"]
    for weather_type in os.listdir(path):

        print(weather_type)
        #joining weather type with main main
        new_path = os.path.join(path,weather_type)
        #joining the path with color_left and depth
        color_left_dir = os.path.join(new_path,"color_left")
        depth_dir = os.path.join(new_path,"depth")

        #looping over every trajectory_directory
        for d1,d2 in zip(os.listdir(color_left_dir), os.listdir(depth_dir)):
            print(d1,d2)
            #joining color_left path with the the current trajectory_directory
            image_directory = os.path.join(color_left_dir, d1)
            #joining depth path with the the current trajectory_directory
            depth_directory = os.path.join(depth_dir, d2)


            #generating 333 random numbers from 0 to amount of images in directory with uniform probability and sorting from low to high
            list_images = sorted(np.random.randint(0,len(os.listdir(image_directory)),image_per_trajectory))
            #print(list_images)

            for i in list_images:

                images = os.listdir(image_directory)
                depth_images = os.listdir(depth_directory)


                x = os.path.join(image_directory,images[i])
                x_test = cv2.imread(x,-1)
                x = cv2.imread(x).astype(float)
                #of size [1,3,256,256]
                x = (resize_256_256(torch.from_numpy(x).unsqueeze(0).permute(0,3,1,2))).squeeze(0)
                combine_images.append(x.half())


                y = os.path.join(depth_directory,depth_images[i])
                #print(y)
                
                
                y = cv2.imread(y, -1).astype("uint16")
                y = y.view("float16")
                #np.clip(y,0.001,255.7500,y)
                

                #[C,W,H]
                y = np.clip(y, 0.001, 80, y)
                y = np.expand_dims(y, axis=2).repeat(3, axis=2)
                
                
                yi = (resize_256_256(torch.from_numpy(y.astype(np.float64)).unsqueeze(0).permute(0,3,1,2))).squeeze(0)
                yi_no_rgb = (resize_256_256(torch.from_numpy(y.astype(np.float64)).unsqueeze(0).permute(0,3,1,2))[:,:1,:,:]).squeeze(0) 
            
                #print(type(yi))
                #print(yi)
                combine_depth_images.append(yi.half())
                combine_depth_images_no_rgb.append(yi_no_rgb.half())
                
                
                
                
                         
        
    tensor_images = torch.stack(combine_images)
    tensor_depth_images = torch.stack(combine_depth_images)
    tensor_depth_images_no_rgb = torch.stack(combine_depth_images_no_rgb)
    
    
    
    
    if not os.path.exists(f"MIDAIR_tensors/{amount_images}_images"):
        os.makedirs(f"MIDAIR_tensors/{amount_images}_images")
        torch.save(tensor_images, f"MIDAIR_tensors/{amount_images}_images/MIDAIR_tensor_images.pt")
        torch.save(tensor_depth_images, f"MIDAIR_tensors/{amount_images}_images/MIDAIR_tensor_depth_images.pt")
        torch.save(tensor_depth_images_no_rgb, f"MIDAIR_tensors/{amount_images}_images/MIDAIR_tensor_depth_images_no_rgb.pt")
    
    del combine_images
    del combine_depth_images
    del combine_depth_images_no_rgb
            
    
    return tensor_images,tensor_depth_images,tensor_depth_images_no_rgb

l = [196,391]
for i in l:
    midair_tensors = create_data_midair(i,main_path)

cloudy
trajectory_3000 trajectory_3000
trajectory_3001 trajectory_3001
trajectory_3002 trajectory_3002
trajectory_3003 trajectory_3003
trajectory_3004 trajectory_3004
trajectory_3005 trajectory_3005
trajectory_3006 trajectory_3006
trajectory_3007 trajectory_3007
trajectory_3008 trajectory_3008
trajectory_3009 trajectory_3009
trajectory_3010 trajectory_3010
trajectory_3011 trajectory_3011
trajectory_3012 trajectory_3012
trajectory_3013 trajectory_3013
trajectory_3014 trajectory_3014
trajectory_3015 trajectory_3015
trajectory_3016 trajectory_3016
trajectory_3017 trajectory_3017
trajectory_3018 trajectory_3018
trajectory_3019 trajectory_3019
trajectory_3020 trajectory_3020
trajectory_3021 trajectory_3021
trajectory_3022 trajectory_3022
trajectory_3023 trajectory_3023
trajectory_3024 trajectory_3024
trajectory_3025 trajectory_3025
trajectory_3026 trajectory_3026
trajectory_3027 trajectory_3027
trajectory_3028 trajectory_3028
trajectory_3029 trajectory_3029
sunny
trajectory_0000 trajectory_